In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap

from tqdm import tqdm


import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

import numba
from numba import jit
import parmap


# 
from track import track
from convert import convert
from ethogram import ethogram

Autosaving every 180 seconds


2021-08-25 16:42:15.251680: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [5]:
# #####################################################
# #####################################################
# #####################################################
class JaabaScores():
    
    def __init__(self, n_animals):

        self.n_animals=n_animals

        self.names = []

        self.scores = []


    def load_scores(self,fname):
        scores = scipy.io.loadmat(fname)
        starts = scores['allScores'][0][0][5][0].squeeze()
        ends = scores['allScores'][0][0][6][0].squeeze()

        #print (starts)
        starts_ends = np.zeros((90000, self.n_animals),'int32')
        for a in range(self.n_animals):
            temp = np.int32((starts[a],ends[a])).squeeze().T
            #print (temp)
            for f in range(temp.shape[0]):
                starts_ends[temp[f][0]:temp[f][1],a] = 1


        #
        self.scores.append(starts_ends)

        #
        name = os.path.split(fname)[1].replace('.mat','').replace('scores_','')
        self.names.append(name)

# 
def plot_vlines(ax,n_chunks, 
                angles_discretized, 
                acc_discretized,
                filter_width, 
                times,
                clrs):
      
#     clrs = [['lightpink','orangered','crimson'],   # left turn
#             ['black','palegreen','green'],    # no turn
#             ['lightblue','deepskyblue','blue']]     # right turn
    
    print ("plotting filter: ", filter_width)

    fps = 25
    plot_width = 1
    for k in range(n_chunks):
        # grab data
        stack = np.int32(np.vstack((angles_discretized[k][2:], acc_discretized[k])).T)

        # filter data
        width = min(filter_width, stack.shape[0])
        stack[:,0] = scipy.ndimage.median_filter(stack[:,0], size=width)
        stack[:,1] = scipy.ndimage.median_filter(stack[:,1], size=width)

        # 
        times_local = np.arange(times[k][0],times[k][1],1)/fps

        for t in range(stack.shape[0]):
            x=times_local[t]
            c = clrs[stack[t,0],stack[t,1]]
            ax.axvspan(x,x+plot_width, 
                       alpha=1, 
                       color=c)


    plt.yticks([])
    plt.xlim(0,x)
        
    return clrs

#
def visualize_discrete_behaviors(fname):
    import matplotlib.patches as mpatches

    clrs = [['lightpink','crimson'],   # left turn
            ['black','green'],    # no turn
            ['lightblue','blue']]     # right turn

            # no mov,     slow,       fast
    clrs = np.array(clrs)

    rows = ['left','straight','right']
    cols = ['no mov','fast']
    
    
    
    d = np.load(fname, allow_pickle=True)
    all_discretized=d['all_discretized']
    angles_thresh=d['angles_thresh']
    angles=d['angles']
    angles_discretized=d['angles_discretized']
    acc_thresh=d['acc_thresh']
    acc=d['acc']
    acc_discretized=d['acc_discretized']
    vel = d['vel']
    times = d['times']

    # 
    print (vel.shape, acc_discretized.shape, angles_discretized.shape)
    print (angles_discretized[0].shape, vel[0].shape, acc_discretized[0].shape)
    print ("accel [0]", acc[0])
    f = 0
    stack = np.vstack((angles_discretized[f][2:], acc_discretized[f])).T
    print (stack.shape, stack)

  

    #################################### 
    #################################### 
    #################################### 

    # offset = 10000
    toffset = 0
    fps=25

    #
    fig =plt.figure()
    ax=plt.subplot(211)
    
    # 
    filter_width=1
    n_chunks = min(50, len(acc_discretized))
    n_chunks = 10
    plot_vlines(ax,n_chunks, 
                angles_discretized, 
                acc_discretized,
                filter_width, 
                times,
                clrs)
        
    #######################################################
    #######################################################
    #######################################################
    ax=plt.subplot(212)
    filter_width=5
    plot_vlines(ax,n_chunks, 
                angles_discretized, 
                acc_discretized,
                filter_width, 
                times,
                clrs)
    
    # where some data has already been plotted to ax
    handles, labels = ax.get_legend_handles_labels()

    for k in range(clrs.shape[0]):
        for p in range(clrs.shape[1]):

            # manually define a new patch 
            patch = mpatches.Patch(color=clrs[k,p], 
                                   label=rows[k]+" "+cols[p])

            # handles is a list, so append manual patch
            handles.append(patch) 

    # plot the legend
    plt.legend(handles=handles, ncol=3)

    # 
    plt.xlabel("Time (sec)")
    plt.ylabel("filter: "+str(filter_width))
    plt.yticks([])
    plt.ylim(0,1)
    
    # 
    plt.show()
    
    
    
##############################################
######### MAKE ANNOTATED MOVIES ##############
##############################################
# 

def make_annotated_video(jb,
                         cohort,
                         filter_width=0
#                          make_movie = True,
#                          start=None,
#                          end=None,
#                          fps=25
                        ):

    
    make_movie = cohort.make_movie
    start = cohort.start
    end = cohort.end
    fps = cohort.movie_output_fps
    #filter_width = cohort.filter_width

    # 
    class_types = cohort.ethogram.class_types
    behaviors = cohort.ethogram.class_names

    # 
    fname_video = cohort.fname_video
    
    # 
    root_name = fname_video.replace('.mp4','').replace('.avi','').replace('.slp','')

    print ("Processing; ", root_name)

    ##########################################
    ##########################################
    ##########################################
    # 
    tracks, classes, angles, acc, class_types = get_classes2(root_name,
                                                             class_types,
                                                             cohort.n_animals, 
                                                             filter_width)

    #
    if make_movie==False:
        return classes, tracks, tracks, angles, acc, behaviors

    # 
    print ("traces: ", tracks.shape)
    print ("classes: ", classes.shape)

    
    #########################################################
    ######## DEFAULT PARMS ######
    #########################################################

    # 
    fname_all_traces = root_name +'.npy'
    print ("fname vidoe: ", fname_video)
    print ("fname_all_traces: ", fname_all_traces)

    # 
    tracks = np.load(fname_all_traces,allow_pickle=True)
    print ("ALL TRACES: ", tracks.shape)    
    
    # 
    colors_rgb = [
                  (0, 0, 255),
                  (255, 0, 0),
                  (255, 255, 0),
                  (0, 128, 0),
                  (255,255,255),
                  (128,0,0),
                  (0,0,128)
                 ]

    #
    names = ['female', 'male', 'pup1', 'pup2']
    clrs = ['blue', 'red', 'cyan', 'green','magenta','pink']  

    # load original video and set settings
    original_vid = cv2.VideoCapture(fname_video)
    ret, frame = original_vid.read()
    size_vid = np.array([frame.shape[1],frame.shape[0]])
    font = cv2.FONT_HERSHEY_PLAIN
    dot_size = 4
           
    # set up output vid
    fname_out = (fname_video.replace('.avi','') + '_' + str(start) + "_" + str(end) + "_"+"filter_"+str(filter_width)+'.mp4')

    # video settings

    # load original vid
    fourcc = cv2.VideoWriter_fourcc('M', 'P', 'E', 'G')
    video_out = cv2.VideoWriter(fname_out,
                                fourcc,
                                fps,
                                (size_vid[0], size_vid[1]),
                                True)

    # reset start of video
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    # loop over frames
    for n in trange(start, end, 1):
        # for n in trange(start, 100, 1):
        ret, frame = original_vid.read()

        # 
        cv2.putText(frame, str(n), (50, 50), font, 5, (255, 255, 0), 5)
        
        
        ############################## 
        ############################## 
        ############################## 
        draw_jaaba_classes(frame, 
                           tracks, 
                           n, 
                           colors_rgb, 
                           font, 
                           jb)
        
        ############################## 
        ############################## 
        ############################## 
        draw_behavior(frame, 
                      tracks, 
                      n, colors_rgb, 
                      font,
                      classes,
                      behaviors)

        ############################## 
        ############################## 
        ############################## 
        draw_features(frame, 
                      tracks, n, 
                      colors_rgb, 
                      font,
                      cohort.ethogram.feature_ids)
        
        # 
        video_out.write(frame)

    # 
    video_out.release()
    original_vid.release()
    
    return classes, tracks, tracks, angles, acc, behaviors
        
# 
def draw_jaaba_classes(frame, 
                       tracks, 
                       n, 
                       colors_rgb, 
                       font, 
                       jb):

    # loop over classes
    for c in range(len(jb.names)):

        # 
        for a in range(tracks.shape[1]):

            # write classes
            behavior = jb.scores[c][n,a]
            if behavior==1:
                #print ("frame: ",n, " animal: ", a, " behavior: ", behavior,

                for f in range(tracks.shape[2]):

                    x = tracks[n, a, f, 0]
                    y = tracks[n, a, f, 1]

                    if np.isnan(x)==True:
                        continue
                    x = int(x)
                    y = int(y)

                    #
                    center_coordinates = (x, y+20)

                    # 
                    behavior = jb.names[c]
                    #color = colors_rgb[a] 
                    color = (0,0,0)
                    frame = cv2.putText(frame,
                                        behavior,
                                        center_coordinates,
                                        font,
                                        2, color, 2)
                    break
    # 
    return frame


def draw_features(frame, 
                  tracks, 
                  n,  
                  colors_rgb, 
                  font,
                  feats):

    # loop over animals 
    for a in range(tracks.shape[1]):
        #color = colors.to_rgb(clrs[a])

        for f in feats:

            x = tracks[n, a, f, 0]
            y = tracks[n, a, f, 1]

            if np.isnan(x) or np.isnan(y):
                continue
                #return frame 
            
            # print ("frame: ", n, " a: ", a, " x, y", x,y, " color: ", color, "clrs: ", clrs[a])

            # 
            x = int(x)
            y = int(y)

            # plot cicrl
            center_coordinates = (x, y)
            radius = 5
            thickness = -1
            color = colors_rgb[a] 
            frame = cv2.circle(frame, 
                               center_coordinates, 
                               radius, 
                               color, 
                               thickness)

          
    return frame
        
# 
def draw_behavior(frame, 
                  tracks, 
                  n,  
                  colors_rgb, 
                  font, 
                  classes,
                  behaviors):
    
    # loop over animals 
    for a in range(tracks.shape[1]):

        # 
        behavior = classes[n,a]
        if np.isnan(behavior)==False:
            
            for f in range(tracks.shape[2]):
                
                x = tracks[n, a, f, 0]
                y = tracks[n, a, f, 1]
                
                if np.isnan(x)==True:
                    continue
                x = int(x)
                y = int(y)
                # plot cicrl
                center_coordinates = (x, y)

                # 
                #print ('n: ', n, " a: ", a, classes.shape, behaviors.shape)
                behavior = behaviors[int(classes[n,a])]
                color = colors_rgb[a] 
                frame = cv2.putText(frame,
                                    behavior,
                                    center_coordinates, 
                                    font, 
                                    1, color, 2)
                break
        #else:
        #    print ("behavior is nan: frame: ", n, " animal: ", a, "  tracks: ",
        #          tracks[n, a])
    # 
    return frame

#
def get_class(stack, class_types):
    
    # 
    classes = np.zeros(stack.shape[0])+np.nan
    # loop over frames
    for k in range(stack.shape[0]):
        temp = stack[k]
        
        # if there are no nans
        if np.any(np.isnan(temp))==False:
            idx = np.sum(np.equal(class_types, temp), axis=1)
            
            # 
            idx2 = np.where(idx==len(temp))[0]
            #if len(idx)>0:
            #print ("k: ", k, " temp: ", temp, " idx1 ",
            #      idx, " idx2: ", idx2)
            classes[k] = idx2
       #print ('')
    return classes


#     
def get_classes2(fname_root, class_types,n_animals, filter_width=5):
    
    # load data        
    fname = fname_root+'_continuous_allData_allPairs.npz'
    d = np.load(fname, allow_pickle=True)
    tracks = d['locs']
    angles_discretized = d['angles_discretized']
    acc_discretized = d['acc_discretized']
    acc_pre = d['acc']
    
        
    # 
#    tracks_smooth = tracks.copy() #np.zeros((tracks.shape[0], tracks.shape[1], 2, 2), 'float32')+np.nan
    classes = np.zeros((tracks.shape[0], tracks.shape[1]), 'float32')+np.nan
#     angles = np.zeros((tracks.shape[0], tracks.shape[1]), 'float32')+np.nan
#     acc = np.zeros((tracks.shape[0], tracks.shape[1]), 'float32')+np.nan
    
    #


    # 
    animal_ids = np.arange(n_animals)
    
    for animal_id in animal_ids:
                    
        angles_animal = angles_discretized[:,animal_id]
        

        acc_animal = acc_discretized[:,animal_id]

        # 
        stack = np.vstack((angles_animal, acc_animal)).T
        
        # compute filter width
        max_width = min(filter_width, stack.shape[0])
            
        # 
        if filter_width>0:
            stack[:,0] = scipy.ndimage.median_filter(stack[:,0], size=max_width)
            stack[:,1] = scipy.ndimage.median_filter(stack[:,1], size=max_width)
            
            for p in range(2):
                for l in range(2):
                    tracks[:, animal_id,p,l] = scipy.ndimage.median_filter(
                                                tracks[:, animal_id,p,l], size=max_width)
            
        # get classes
        class_local = get_class(stack, class_types)
        classes[:, animal_id] = class_local

        
    return tracks, classes, angles_discretized, acc_discretized, class_types

In [6]:
class Cohort():

    def __init__(self, fname):
        
        #
        self.n_cores = 8

        #
        self.fname_slp = fname
        
        #
        self.root_dir = os.path.split(fname)[0]
                
            
        # make track object
        self.trk = track.Track(self.fname_slp)

        # make ethogram object    
        self.ethogram = ethogram.Ethogram(self.fname_slp)

                
            
    def process(self):
        
        #
        self.process_slp()
        
        # 
        self.process_ethograms()
        

    def process_slp(self):

        # 
        self.trk.verbose = False
        self.trk.max_jump_single_frame = 100  # max distance that an animal can move in a single frame
        self.trk.max_time_automerge = 3
        self.trk.max_dist_automerge = 25       # distance to auto merge chunks from same animal separated by single time skip
        self.fdir_out = self.fname_slp.replace('.slp','')

        #self.trk.n_animals = 6

        if self.trk.n_animals==4:
            self.trk.tracks_names = ['female', 'male', 'pup shaved', 'pup unshaved']
        else:
            self.trk.tracks_names = ['female', 'male', 'pup1','pup2','pup3','pup4']
            

        # convert data to tracks
        self.trk.load_tracks()
        
        # 
        if True:
            
            # 
            self.trk.get_scores()

            #
            self.trk.make_tracks_chunks()

            # 
            min_chunk_len=5
            self.trk.del_short_chunks(min_chunk_len)

            # 
            self.trk.merge_single_jumps()

            # 
            self.fix_tracks_memory_filter()
            
        print ("Done processing slp")
            
    #    
    def fix_tracks_memory_filter(self):
    
        #
        print ("fixing track w. memory filter")

        # 
        fname_out_spine = self.trk.fname_slp.replace('.slp',"_spine_fixed.npy")
        fname_out = self.trk.fname_slp.replace('.slp',"_fixed.npy")

        # TODO MOVE THESE FUNCTIONS INSIDE track class
        #if os.path.exists(fname_out_spine)==False:
        if True: 
            # 
            start = 10
            end = self.trk.tracks_spine.shape[0]

            # 
            self.trk.time_threshold = 25       # window to search for nearest chunks
            self.trk.safe_chunk_length = 15    # chunks this long will not change id
            self.trk.min_chunk_len = 4         # min length of chukn to be used for anchoring/correcting
            self.trk.max_distance_merge = 75  # max pix diff allowed for merging when using model
            # track.memory_length = 25      # how many frames back is it ok to remember a prev animal

            # 
            self.trk.verbose = False
            self.trk.update_tracks = True

            #
            if True:
                self.trk.fix_tracks(start, end)

                # save tracks
                np.save(fname_out, self.trk.tracks_fixed)
                np.save(fname_out_spine, self.trk.tracks_spine_fixed)


    def process_ethograms(self):
        
        # 
        
        # 
        self.ethogram.min_duration = 3  # minimum # of ???
                
        # 
        self.ethogram.n_animals = self.trk.n_animals
        
        #
        if self.ethogram.n_animals==4: # 4 animal recordings
            self.ethogram.feature_ids = np.array([0,5,6,7,8,9])            
        elif self.ethogram.n_animals==6:     # 6 animal recordings
            self.ethogram.feature_ids = np.arange(6)

        # 
        self.ethogram.compute_discretized_and_histograms_continuous_all_pairs()
        
        print ('')
        print ('')
        print ('')


def plot_class_rasters(classes,
              start,
              end):

    import matplotlib.patches as mpatches
    
    clrs = ['lightpink','crimson',   # left turn
            'black','green',    # no turn
            'lightblue','blue']     # right turn

            # no mov,     slow,       fast
    clrs = np.array(clrs)

    rows = ['left','straight','right']
    cols = ['no mov','fast']
    
    class_names = [
        'left ',
        'left run',
        'straight ',
        'straight run',
        'right ',
        'right run'
    ]
    
    
    plot_width=1.0/25
    ax=plt.subplot(1,1,1)
    for a in range(classes.shape[1]):
        #ax=plt.subplot(6,1,a+1)
        for k in range(start, end, 1):
            
            x=classes[k, a]
            if np.isnan(x)==False:
                x = int(x)
                c = clrs[x]
                ax.axvspan(k/25,k/25+plot_width, 
                           ymin = a/classes.shape[1],
                           ymax = (a+1)/classes.shape[1],
                           alpha=1, 
                           color=c)
            
        #plt.xticks([])
    
    plt.xlim(start/25.,end/25.)
    plt.ylabel("Animal ID ")
    #plt.yticks([])
    plt.xlabel("Time (sec)")
    yticks = np.arange(classes.shape[1])
    plt.yticks(np.linspace(0, 1, classes.shape[1]+1)[1:]-1./classes.shape[1]/2.,
               yticks)
    #plt.xticks([])
    
    # manually define a new patch 
    handles=[]
    for k in range(len(clrs)):
        patch = mpatches.Patch(color=clrs[k], 
                           label=class_names[k])

        # handles is a list, so append manual patch
        handles.append(patch) 

    # plot the legend
    plt.legend(handles=handles, ncol=3)

    
    
    plt.show()
    

# 
def get_durations(array, 
                  n_times):
    
    # 
    durations = []
    starts = []
    ends = []
    starts.append(array[0])
    
    
    # keep track of how many frames inside in order to append to end of data
    inside_ctr = 0
    for f in range(0,array.shape[0]-1,1):
        
        #
        if array[f+1]-array[f]>1:
            
            # 
            ends.append(array[f]+1)
            starts.append(array[f+1])
            inside_ctr=0
        else:
            inside_ctr+=1
    
    ends.append(starts[-1]+inside_ctr+1)
    
    #
    a = np.vstack((starts,ends)).T
    
    dur = a[:,1]-a[:,0]
    
    return dur
    

def plot_summary_bar_histograms(class_names, behaviors, classified_time):
    #
    clrs = ['lightpink','crimson',   # left turn
            'black','green',    # no turn
            'lightblue','blue',
            'lightgreen']     # right turn

    # 
    clrs = np.array(clrs)

    class_names = [
        'left ',
        'left run',
        'straight ',
        'straight run',
        'right ',
        'right run'
    ]

    # 
    class_names = behaviors+[" unclassified"]
    width = 0.1*.9
    for k in range(classified_time.shape[0]):
        ax=plt.subplot(2,2,k+1)
        plt.xticks([])
        plt.ylabel("Time(s)")
        plt.title(animal_names[k])

        for p in range(classified_time.shape[1]):

            plt.bar(p*.1, classified_time[k,p], 
                    width,
                    color=clrs[p],
                    label=class_names[p])

        if k==0:
            plt.legend()

        #plt.semilogy()
        plt.ylim(1E-1,3E2)

    plt.suptitle("300sec dataset analysis (5mins or 7500 frames)")
    plt.show()

    

def plot_behavior_histograms(animal_ids, 
                             classes, 
                             n_classes,
                             plotting=False):
    
    # 
    animal_ids = np.arange(classes.shape[1])
    
    # n_animals, n_behaviors + the nan class
    classified_time = np.zeros((classes.shape[1], n_classes+1),'float32')
    
    movement_bouts = []
    data_bouts = []
    
    #
    fps = 25.
    bin_width = 1
    bins = np.arange(0,100000,bin_width)

    # 
    for animal_id in animal_ids:
        if plotting:
            fig=plt.figure(figsize=(7.5,5))
        
        # 
        classes_animal = classes[:, animal_id]

        tot_time = 0
        for k in range(n_classes):

            # 
            n_times = classes_animal.shape[0]
            idx = np.where(classes_animal==k)[0] 
            durations = get_durations(idx, n_times)
            tot_time += np.sum(durations)/fps
                        
            # 
            classified_time[animal_id, k] = np.sum(durations)/fps

            #
            idxnan = np.where(np.isnan(classes_animal))[0]
            dur_nan = get_durations(idxnan, n_times)
            dur_nan = np.sum(dur_nan)
            classified_time[animal_id, -1] = dur_nan/fps

            # 
            y = np.histogram(durations, bins = bins)

            # 
            x = y[1][1:]/fps
            y = y[0]
            
            if plotting:
                ax=plt.subplot(2,3,k+1)

                plt.plot(x,y, c='blue',
                        linewidth=linewidth)
                plt.title(behaviors[k])
                plt.semilogy()
                plt.semilogx()
                if k%3==0:
                    ax.set_ylabel("# behaviors")

            ################################
            ################################
            ################################
            if plotting:
                ax2 = ax.twinx()
            
            # 
            cumsum = np.cumsum(y*x)

            # 
            if plotting:
                ax2.plot(x,cumsum, c='red',
                        linewidth=linewidth)
                if k%3==2:
                    ax2.set_ylabel("# seconds in bheavior")

                #     
                plt.ylim(bottom=0)
                plt.xlim(left=1E-1)

                # 
                if k>=3:
                    ax.set_xlabel("Time (s)")

        # compute bouts where behavior is not nan or 2 (i.e. no moving straight)
        idx_nonnan = np.where(np.isnan(classes_animal)==False)[0]
        # idx_mov = np.where(classes_animal[idx_nonnan]!=2)[0]  # USE THIS FOR 2 categories, i.e. no-mov / run splits
        idx_mov = np.where(classes_animal[idx_nonnan]!=3)[0]
        
        # idx_bouts = np.intersect1d(idx_nonnan, idx_mov)
        idx_bouts = idx_nonnan[idx_mov]
        
        # 
        dur_bouts = get_durations(idx_bouts,n_times)
        #print ("mov bouts: ", idx_bouts)
        
        movement_bouts.append(dur_bouts)
        
        # data bouts
        d_bouts = get_durations(idx_nonnan, n_times)
        data_bouts.append(d_bouts)

        # 
        if plotting:

            plt.suptitle(animal_names[animal_id]+ " labeled: "+str(round(tot_time,2))+ 
                         " (unlabeled "+ str(round(dur_nan/fps,2)) + "sec)")
            print ("tottime: ", tot_time)

            if False:
                fig.savefig('/home/cat/histogram_'+str(animal_id)+'.png',dpi=300)
                plt.close()
            else:
                plt.show()
    
    return classified_time, movement_bouts, data_bouts

# 
def plot_bout_histograms(cohort,
                         bout_type,
                         jb):
    
# def plot_bout_histogram(bout_type,
#                         fname_video, 
#                         jb):


    # 
    fname_video = cohort.fname_video
    
    n_animals = jb.n_animals
    animal_ids = np.arange(n_animals)

    # 
    clrs = ['lightpink','crimson',   # left turn
            'black','green',    # no turn
            'lightblue','blue',
            'lightgreen']     # right turn

    # 
    fps = 5

    make_movie = False

    #     
    for k in range(n_animals):
        plt.xlabel("Duration of behavior bout (s)")

        # 
        filter_widths = [0,3,5,7]
        max_y = 0
        ax=plt.subplot(2,2,k+1)
        ax2 = ax.twinx()
        for ctr,filter_width in enumerate(filter_widths):

            ##################################
            ###### GET FILTERED CLASSES ######
            ##################################
            fps = 25
            start = 0*fps
            end = start+100
            make_movie = False
            classes, tracks, tracks_smooth, angles, acc, behaviors = make_annotated_video(jb,
                                                                                          cohort)
            
            ##################################
            ##################################
            ##################################
            n_classes = 6
            bin_width = 1
            linewidth=3
            # 
            plotting = False
            _, behavior_bouts, data_bouts = plot_behavior_histograms(animal_ids, classes, n_classes, plotting)

            ##################################
            ######## GENERATE CURVES #########
            ##################################
            
            if bout_type == 'movement':
                bouts = behavior_bouts
                temp = np.float32(bouts[k])/fps
                width = 1
                bins = np.arange(-1.0,1*fps,width)/fps
                print ("BINS: ", bins)

                ax.plot([0.1,0.1],[0,30],'--',c='grey')    
                ax.text(0.1,30, '100ms')
                ax.plot([0.04,0.04],[0,40],'--',c='grey')    
                ax.text(0.04,40, '40ms (single frame)')


                
            else:
                bouts = data_bouts
                
                temp = np.float32(bouts[k])/fps
                width = 25
                bins = np.arange(-1.0,tracks.shape[0],width)/fps
                print ("BINS: ", bins)
                ax.semilogx()


            y = np.histogram(temp,bins)
            x = y[1][1:]
            y = y[0]
            ax.plot(x,y,
                    c=clrs[ctr],
                    linewidth=3,
                    label=str(filter_width))

            max_y = max(np.max(y),max_y)
            #ax.set_xlim(left=0)
            #ax.set_ylim(0, max_y)

            ####################################
            ######## TIWN AX CUMSUM ############
            ####################################
            #
            if filter_width==3 or filter_width==0:

                # 
                cumsum = np.cumsum(y*x)

                # 
                ax2.plot(x,cumsum, '--',c=clrs[ctr],
                        linewidth=linewidth)
                ax2.set_ylabel("# seconds in bheavior (total = 300s)")

                #     
                ax2.set_ylim(bottom=0)
                ax2.set_xlim(left=0)



        #
        ax.set_xlabel("Duration of bout (s)")
        ax.set_title(animal_names[k])
        ax.set_ylim(bottom=0)
        ax.set_xlim(left=0)
        ax.set_ylabel("# of bouts")

        if k==0:
            ax.legend(title='filter width', fontsize=12)

    plt.show()
    
# 
def plot_bout_histograms_multi_day(fnames_slp,
                                   cohort,
                                   bout_type,
                                   filter_width,
                                   jb):
    linewidth = 5
    clrs = ['black','blue','red','magenta']
    animal_names = ['female','male','pup1','pup2','pup3','pup4']

    # 
    n_animals = jb.n_animals
    animal_ids = np.arange(n_animals)
    make_movie = False
    #jb.make_movie = False
    cohort.make_movie = False
    
    # make lists to hold data
    #filter_width = 0
    fps = 25
    start = 0*fps
    end = start+100
    make_movie = False

    # loop over all fnames
    behavior_bouts_all = []
    data_bouts_all = []
    for fname_slp in tqdm(fnames_slp):
    
        fname_out_bouts = fname_slp.replace('.slp','_bouts_filter_'+str(filter_width)+'.npz')

        if os.path.exists(fname_out_bouts)==False:
            cohort.fname_video = fname_slp.replace('.slp','.avi')


            #     
            ##################################
            ###### GET FILTERED CLASSES ######
            ##################################
            classes, tracks, tracks_smooth, angles, acc, behaviors = make_annotated_video(jb,
                                                                                          cohort)

            ##################################
            ##################################
            ##################################
            n_classes = 6
            bin_width = 1
            linewidth = 3
            # 
            plotting = False

            # this returns duration of segments of bouts
            _, behavior_bouts, data_bouts = plot_behavior_histograms(animal_ids, classes, n_classes, plotting)

            # 
            np.savez(fname_out_bouts,
                     behavior_bouts = behavior_bouts,
                     data_bouts = data_bouts)
            
            behavior_bouts_all.append(behavior_bouts)
            data_bouts_all.append(data_bouts)
        
        # 
        else:
            
            data = np.load(fname_out_bouts, allow_pickle=True)
            behavior_bouts_all.append(data['behavior_bouts'])
            data_bouts_all.append(data['data_bouts'])
            
            
    ##################################
    ######## GENERATE CURVES #########
    ##################################
#     print ("behavior_bouts_all[0]", 
#            len(behavior_bouts_all[0]), 
#            len(behavior_bouts_all[0][0]))
           
    behavior_bouts_all = np.array(behavior_bouts_all)
    data_bouts_all = np.array(data_bouts_all)

    for k in range(n_animals):
        max_y = 0
        ax=plt.subplot(2,2,k+1)
        ax2 = ax.twinx()
        
        #
        
        behavior_bouts = np.hstack(behavior_bouts_all[:,k])
        print ("animaL: ", k, "behavior_bouts ",behavior_bouts )
        
        data_bouts = np.hstack(data_bouts_all[:,k])
        if bout_type == 'movement':
            temp = np.float32(behavior_bouts)/fps
            width = 1
            bins = np.arange(-1.0,100*fps,width)/fps
            #print ("BINS: ", bins)
            ax.semilogx()
            ax.semilogy()

        else:
            temp = np.float32(data_bouts)/fps
            width = 25
            bins = np.arange(-1.0,100000,width)/fps
            #print ("BINS: ", bins)
            ax.semilogx()
            ax.semilogy()


        y = np.histogram(temp,bins)
        x = y[1][1:]
        y = y[0]
        ax.plot(x,y,
                c='black',
                linewidth=3,
                label=str(filter_width))

        max_y = max(np.max(y),max_y)
        #ax.set_xlim(left=0)
        #ax.set_ylim(0, max_y)
        
        if bout_type == 'movement':
            
            ax.plot([0.1,0.1],[0,max_y*.9],'--',c='grey')    
            ax.text(0.1,max_y*.9, '100ms')
            ax.plot([0.04,0.04],[0,max_y*.7],'--',c='grey')    
            ax.text(0.04,max_y*.7, '40ms (single frame)')
        
        
        ####################################
        ######## TIWN AX CUMSUM ############
        ####################################
        #
        if filter_width==3 or filter_width==0:

            # 
            cumsum = np.cumsum(y*x)

            # 
            ax2.plot(x,cumsum, '--',c='red',
                    linewidth=linewidth)
            if k%2==1:
                ax2.set_ylabel("Total # seconds of data")

            #     
            ax2.set_ylim(bottom=0)
            ax2.set_xlim(left=0)

        #
        if k>=2:
            ax.set_xlabel("Duration of bout (s)")
        ax.set_title(animal_names[k])
        ax.set_ylim(bottom=0)
        ax.set_xlim(left=0)
        ax.set_ylabel("# of bouts")

        if k==0:
            ax.legend(title='filter width', fontsize=12)
    plt.suptitle("Filter: "+ str(filter_width))
    plt.show()
    

In [7]:
##############################################
######### RUN .SLP -> ANNOTATIONS CODE #######
##############################################

# 

def cohort_object_initialize(fname):
    
    # 
    cohort = Cohort(fname)
    
    # 
    if 'cohort1' in fname:
        cohort.trk.n_animals = 4               # TO AUTODETECT
    else:
        cohort.trk.n_animals = 6
    
    # 
    cohort.ethogram.fixed_tracks_flag = False   # uses the track memory fixed data for downstream
    
    # 
    cohort.ethogram.smooth_tracks_flag = False   # currently not working well

    #
    cohort.ethogram.smooth_angles_flag = False   # currently not working well

        
    #
    #cohort.ethogram.features_anchor = np.array([0,1])
    # cohort.ethogram.features_anchor = np.array([0,3])
    #cohort.ethogram.features_anchor = np.array([1,2])
    cohort.ethogram.features_anchor = np.array([2,3])  # THIS IS NOT USED ANYMORE FOR ALL PAIR DISCRETIZATION
    
    #
    cohort.ethogram.min_rot = 20        # all values below 20 are reduced to zero and not added to the cumulative sum
                                        # in principle this should denoise the jitters
                                        # not clear this does anything as only >45Degree data matters

    #
    cohort.ethogram.replace_nans_with_last_loc = True      # this replaces missing Sleap Nans with last value found;
    
        
    # 
    cohort.ethogram.cumulative_sum = True     # this does running cumulative measure until +/- 45 degree
                                                    # movements are observed and then it zeros out again
    
    min_angle = 90
    cohort.ethogram.angles_thresh = [[-1E8, -min_angle],
                                     [-min_angle,+min_angle],
                                     [+min_angle,1E8]
                                    ]
    
    #
    cohort.ethogram.walk_run_flag = True
    
    if cohort.ethogram.walk_run_flag==False: 
        cohort.ethogram.acc_thresh = [[0,130],
                                      [130,1E8]]
    
        cohort.ethogram.class_names = [
                                        'left ',
                                        'left run',
                                        'straight ',
                                        'straight run',
                                        'right ',
                                        'right run'
                                        ]
            
        # 
        class_types = []
        # loop over angle
        for a in range(3):
            # loop over speed
            for s in range(2):
                class_types.append([a,s])

        # 
        cohort.ethogram.class_types = np.array(class_types)

        
    else:
        # 
        walk_thresh = 115
        run_thresh = 230
        cohort.ethogram.acc_thresh = [[0,walk_thresh],
                                     [walk_thresh, run_thresh],
                                     [run_thresh,1E8]]
        
        # 
        cohort.ethogram.class_names = [
                                        'left ',
                                        'left walk',
                                        'left run',
                                        'no mov ',
                                        'walk',
                                        'run',
                                        'right ',
                                        'right walk',
                                        'right run'
                                      ]

        # 3 angles, 3 runs
        class_types = []
        for a in range(3):
            for s in range(3):
                class_types.append([a,s])
        cohort.ethogram.class_types = np.array(class_types)
        
        cohort.ethogram.clrs =  ['lightpink','crimson', 'red',    # left turn
                                   'black','grey','green',        # no turn
                                   'lightblue','azur','blue']     # right turn

        
    return cohort

def run_cohort_parallel(fname):

    cohort = Cohort(fname)
    
    # 
    if 'cohort1' in fname:
        cohort.trk.n_animals = 4               # TO AUTODETECT
    else:
        cohort.trk.n_animals = 6
    
    # 
    cohort.ethogram.fixed_tracks_flag = True   # uses the track memory fixed data for downstream
    
    # 
    cohort.ethogram.smooth_tracks_flag = False   # currently not working well

    #
    cohort.ethogram.smooth_angles_flag = False   # currently not working well

        
    #
    #cohort.ethogram.features_anchor = np.array([0,1])
    # cohort.ethogram.features_anchor = np.array([0,3])
    #cohort.ethogram.features_anchor = np.array([1,2])
    cohort.ethogram.features_anchor = np.array([2,3])  # THIS IS NOT USED ANYMORE FOR ALL PAIR DISCRETIZATION
    
    #
    cohort.ethogram.min_rot = 20        # all values below 20 are reduced to zero and not added to the cumulative sum
                                        # in principle this should denoise the jitters
                                        # not clear this does anything as only >45Degree data matters

    #
    cohort.ethogram.replace_nans_with_last_loc = True      # this replaces missing Sleap Nans with last value found;
    
        
    # 
    cohort.ethogram.cumulative_sum = True     # this does running cumulative measure until +/- 45 degree
                                                    # movements are observed and then it zeros out again
    
    min_angle = 90
    cohort.ethogram.angles_thresh = [[-1E8, -min_angle],
                                     [-min_angle,+min_angle],
                                     [+min_angle,1E8]
                                    ]
    
    #
    cohort.ethogram.walk_run_flag = True
    
    if cohort.ethogram.walk_run_flag==False: 
        cohort.ethogram.acc_thresh = [[0,130],
                                      [130,1E8]]
    
        cohort.ethogram.class_names = [
                                        'left ',
                                        'left run',
                                        'straight ',
                                        'straight run',
                                        'right ',
                                        'right run'
                                        ]
            
        # 
        class_types = []
        # loop over angle
        for a in range(3):
            # loop over speed
            for s in range(2):
                class_types.append([a,s])

        # 
        cohort.ethogram.class_types = np.array(class_types)

        
    else:
        # 
        walk_thresh = 115
        run_thresh = 230
        cohort.ethogram.acc_thresh = [[0,walk_thresh],
                                     [walk_thresh, run_thresh],
                                     [run_thresh,1E8]]
        
        # 
        cohort.ethogram.class_names = [
                                        'left ',
                                        'left walk',
                                        'left run',
                                        'no mov ',
                                        'walk',
                                        'run',
                                        'right ',
                                        'right walk',
                                        'right run'
                                      ]

        # 3 angles, 3 runs
        class_types = []
        for a in range(3):
            for s in range(3):
                class_types.append([a,s])
        cohort.ethogram.class_types = np.array(class_types)
        
        cohort.ethogram.clrs =  ['lightpink','crimson', 'red',    # left turn
                                   'black','grey','green',        # no turn
                                   'lightblue','azur','blue']     # right turn

    #
    cohort.process()

    

In [8]:
############################################
############################################
############################################

# 
#fnames = np.loadtxt('/media/cat/1TB/dan/cohort1/slp.txt',dtype=np.str)
#fnames = np.loadtxt('/media/cat/1TB/dan/cohort1/march16/march16_file_list.txt',dtype='str')

fnames = [
    '/media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_01_57_27_327194_compressed_26min_28min.slp',
'/media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_04_58_11_998956_compressed_46min_48min.slp'
]

if True:
    parmap.map(run_cohort_parallel,
               fnames,
               pm_processes=6,
               pm_pbar=True)
else:
    pass
# 

  0%|                                                                            | 0/2 [00:00<?, ?it/s]

fixing track w. memory filterfixing track w. memory filter

UPDATING TRACKS
UPDATING TRACKS
Done processing slp
... DONE...

Done processing slp


... DONE...





100%|████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 46.52it/s]


In [9]:
###########################################
########## MAKE ANNOTATED VIDEOS ##########
###########################################
import cv2
from matplotlib import colors

fname = '/media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_01_57_27_327194_compressed_26min_28min.slp'

#
cohort = cohort_object_initialize(fname)


# SET VIDEO PARAMS
if 'cohort2' in fname:
    cohort.n_animals = 6
else:
    cohort.n_animals = 4
    
cohort.start = 0
cohort.end = cohort.start+120*25
cohort.make_movie = True
cohort.fname_video = cohort.fname_slp.replace('.slp','.avi')
cohort.filter_width = 0
cohort.movie_output_fps = 5


# ADD JAABA SCORES IF AVAILABLE
jb = JaabaScores(cohort.n_animals)

if False:
#if 'cohort1' in fname:
    jb.load_scores('/media/cat/1TB/dan/cohort1/2020-3-8_0day_5mins_compressed/scores_approaching.mat')
    jb.load_scores('/media/cat/1TB/dan/cohort1/2020-3-8_0day_5mins_compressed/scores_moving_away.mat')

print ("jb na nimal: ",jb.n_animals)

# run movie maker
classes, tracks, tracks_smooth, angles, acc, behaviors = make_annotated_video(jb,
                                                                              cohort)




jb na nimal:  4
Processing;  /media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_01_57_27_327194_compressed_26min_28min
traces:  (3000, 4, 14, 2)
classes:  (3000, 4)
fname vidoe:  /media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_01_57_27_327194_compressed_26min_28min.avi
fname_all_traces:  /media/cat/1TB/dan/cohort1/march_16_repredicted/2020_3_16_01_57_27_327194_compressed_26min_28min.npy
ALL TRACES:  (3000, 4, 14, 2)


OpenCV: FFMPEG: tag 0x4745504d/'MPEG' is not supported with codec id 2 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████████████████████████████████████████████████████████| 3000/3000 [00:34<00:00, 86.33it/s]


In [5]:
##############################################
########### SHOW BEHAVIOR RASTERS ############
##############################################
print ("CLASS TYPES: ", class_types)
    
fig = plt.figure()
start =800
end = start+200
plot_class_rasters(classes, start, end)

CLASS TYPES:  [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]


NameError: name 'classes' is not defined

In [28]:
##############################################
########## HISTOGRAMS OF DURATIONS ###########
##############################################
animal_names = ['female','male','pup1','pup2','pup3','pup4']
print ("behaviors: ", behaviors)
n_classes = 6
bin_width = 1
linewidth=3
animal_ids = np.arange(cohort.ethogram.n_animals)

class_names = cohort.ethogram.class_names

# 
plotting = False
classified_time, movement_bouts, data_bouts = plot_behavior_histograms(animal_ids, classes, n_classes, plotting)

#
plot_summary_bar_histograms(class_names, behaviors, classified_time)


behaviors:  ['left', 'left run', 'straight', 'straight run', 'right', 'right run']
idxbouts:  [  30   31   35   36   37   38   58   59   60   61   62   63   64   65
   66   77   78   79   80   81   82   83   84   85   90   99  100  101
  102  103  104  107  108  109  117  118  123  124  125  126  127  128
  129  130  131  132  133  136  137  138  139  140  141  142  143  144
  145  146  147  148  149  150  151  152  153  154  157  158  162  163
  164  165  166  175  176  177  178  179  180  181  182  183  184  185
  189  190  195  196  208  209  210  211  212  213  214  215  216  217
  218  221  222  223  304  305  306  307  308  309  325  326  327  328
  329  330  331  332  333  335  336  337  338  339  340  341  399  401
  402  464  465  466  488  489  490  491  492  493  494  495  496  497
  498  499  500  501  502  503  504  505  506  507  508  509  510  511
  512  513  514 1056 1057 1058 1068 1069 1139 1140 1141 1142 1143 1144
 1145 1146 1147 1148 1149 1150 1151 1152 1153 1156 115

In [30]:
###################################################
####### MOVEMENT OR DATA BOUTS HISTOGRAMS #########
###################################################

# 
bout_type = 'movement'  # types: 'movement', 'data'
plot_bout_histograms(cohort,
                     bout_type,
                     fname_video, 
                     jb)
            

fname vidoe:  /media/cat/1TB/dan/cohort1/2020-3-8_0day_5mins_compressed/2020-3-8_0day_5mins_compressed.mp4
fname_all_traces:  /media/cat/1TB/dan/cohort1/2020-3-8_0day_5mins_compressed/2020-3-8_0day_5mins_compressed.npy
ALL TRACES:  (7500, 4, 14, 2)
traces:  (7500, 4, 14, 2)
classes:  (7500, 4)
idxbouts:  [  10   29   30 ... 7422 7470 7473]
idxbouts:  [   6    8    9 ... 7369 7394 7413]
idxbouts:  [  59   60   61   62   63   64   65   66   67   68   69   70   72   73
   74   75   76   77   79   80   81   82   83   85   86   87   90   91
   92  113  172  173  197  242  435  498  539  540  543  544  584  585
  586  587  588  589  590  592  595  596  631  632  634  635  636  637
  638  639  640  641  642  644  647  652  653  654  656  658  659  660
  664  665  666  667  668  669  670  671  672  673  674  675  676  679
  683  684  685  686  694  695  696  703  717  720  726  727  728  729
  731  747  748  749  750  751  752  754  755  774  775  776  777  794
  799  800  801  829  845  849  

In [109]:
###################################################
####### MOVEMENT OR DATA BOUTS HISTOGRAMS #########
###################################################

fnames = np.loadtxt('/media/cat/1TB/dan/cohort1/march16/march16_file_list.txt',
                   dtype='str')
for ctr, fname in enumerate(fnames):
    print (ctr, fname)
cohort = cohort_object_initialize(fnames[0])
#
jb = JaabaScores(cohort.trk.n_animals)

#
cohort.filter_width = 0
cohort.n_animals = 4
cohort.start = 0
cohort.end = 100
cohort.movie_output_fps = 5
cohort.fname_video = fnames[0]

# 
bout_type = 'movement'  # types: 'movement', 'data'
fig=plt.figure()
fname_in = [fnames[7]]
print ("fname in :", fname_in)
plot_bout_histograms_multi_day(fname_in,
                               cohort,
                               bout_type,
                               cohort.filter_width,
                               jb)
plt.suptitle(fname_in)
plt.show()

0 /media/cat/1TB/dan/cohort1/march16/2020_3_16_01_54_23_358257_compressed.slp
1 /media/cat/1TB/dan/cohort1/march16/2020_3_16_01_57_27_327194_compressed.slp
2 /media/cat/1TB/dan/cohort1/march16/2020_3_16_02_54_39_170978_compressed.slp
3 /media/cat/1TB/dan/cohort1/march16/2020_3_16_02_57_41_995158_compressed.slp
4 /media/cat/1TB/dan/cohort1/march16/2020_3_16_03_54_54_231226_compressed.slp
5 /media/cat/1TB/dan/cohort1/march16/2020_3_16_03_57_56_902379_compressed.slp
6 /media/cat/1TB/dan/cohort1/march16/2020_3_16_04_55_09_841582_compressed.slp
7 /media/cat/1TB/dan/cohort1/march16/2020_3_16_04_58_11_998956_compressed.slp
8 /media/cat/1TB/dan/cohort1/march16/2020_3_16_05_55_25_305681_compressed.slp
9 /media/cat/1TB/dan/cohort1/march16/2020_3_16_05_58_27_193818_compressed.slp
10 /media/cat/1TB/dan/cohort1/march16/2020_3_16_06_55_40_714236_compressed.slp
11 /media/cat/1TB/dan/cohort1/march16/2020_3_16_06_58_43_678014_compressed.slp
12 /media/cat/1TB/dan/cohort1/march16/2020_3_16_07_55_55_77523

  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing;  /media/cat/1TB/dan/cohort1/march16/2020_3_16_04_58_11_998956_compressed


100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.37s/it]

animaL:  0 behavior_bouts  [1 2 1 ... 4 1 1]
animaL:  1 behavior_bouts  [1 3 1 ... 7 1 2]
animaL:  2 behavior_bouts  [3 3 2 ... 2 1 1]
animaL:  3 behavior_bouts  [1 1 1 ... 1 1 2]


In [26]:
data = np.load('/media/cat/1TB/dan/cohort1/march16/2020_3_16_03_54_54_231226_compressed_continuous_allData_allPairs.npz', allow_pickle=True)
locs = data['locs']
print ('locs: ', locs.shape)
print (np.any(np.isnan(locs)))

acc = data['acc_discretized']
angles = data['angles_discretized']

print (acc)
print (np.any(np.isnan(acc[2:])))

print (np.any(np.isnan(angles[2:-2])))


locs:  (89987, 4, 14, 2)
False
[[nan nan nan nan]
 [nan nan nan nan]
 [ 0.  0.  0.  0.]
 ...
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
False
False


In [72]:
#############################
######## SHOW CLASSES #######
#############################
print ("classes: ", classes.shape)

t=np.arange(classes.shape[0])/25.

fit = plt.figure()

ax=plt.subplot(2,2,1)
for k in range(classes.shape[1]):
    plt.plot(t,classes[:,k]+k*10)

ax=plt.subplot(2,2,2)
for k in range(classes.shape[1]):
    plt.plot(t,angles[:,k]+k*10)

ax=plt.subplot(2,2,3)
for k in range(classes.shape[1]):
    plt.plot(t,acc[:,k]+k*10)

plt.show()

classes:  (28802, 6)


In [29]:
#############################
######## SHOW TRACKS #######
#############################
clrs = ['lightpink','crimson',   # left turn
        'black','green',    # no turn
        'lightblue','blue']     # right turn

fig = plt.figure()
print (tracks.shape, tracks_smooth.shape)
t=np.arange(tracks.shape[0])/25.
t2=np.arange(tracks_smooth.shape[0])/25.
for k in range(tracks.shape[1]):
    plt.plot(t,tracks[:,k,0,0]+k*1000,c=clrs[k])
    #plt.plot(t,tracks[:,k,1,0]+k*1010,c=clrs[k])
    plt.plot(t2,tracks_smooth[:,k,0,0]+k*1100,
             linewidth = 3, 
             c=clrs[k])

    
plt.show()

(28802, 6, 6, 2) (90000, 6, 2, 2)


In [73]:
####################################
######## SHOW VSPAN RASTERS ########
####################################
#
fname = '/media/cat/1TB/dan/cohort1/slp_files/2020_3_16_01_54_23_358257_compressed_animalID_0_single_frame_allData.npz'

# 
visualize_discrete_behaviors(fname)

    

(562,) (562,) (562,)
(7,) (6,) (5,)
accel [0] [5275.2695  2592.4165   103.35638  101.26182  161.87811]
(5, 2) [[1. 1.]
 [0. 1.]
 [2. 1.]
 [2. 1.]
 [2. 1.]]
plotting filter:  1
plotting filter:  5


In [40]:
data = np.load('/media/cat/1TB/dan/cohort2/2020_07_21_11_26_07_855478/2020_07_21_11_26_07_855478_compressed_continuous_allData.npz',
              allow_pickle=True)

angles_discretized = data['angles_discretized']

for k in range(6):
    print (angles_discretized.shape)
    plt.plot(angles_discretized[:,k]+k*100)
plt.show()

(28802, 6)
(28802, 6)
(28802, 6)
(28802, 6)
(28802, 6)
(28802, 6)


In [19]:
######################################
######## PREPROCESS DATA #############
######################################
animal_id = 0
# min_duration = 25

In [139]:
#####################################
######### PREPROCESS DATA ###########
#####################################

compute_discretized_and_histograms_single_frame(animal_id)

agnels thresholds:  [[-100000000.0, -45], [-45, 45], [45, 100000000.0]]
accelaration thresholds:  [[0, 40], [40, 100000000.0]]
 FRAME IDS MISSING , UPGRADE DATAFILES 
ANIMAL Specific data:  (2069710, 6, 2)
# of headnose locations:  (521682,)
HEADNOSE:  [     13      24      32 ... 2063791 2063792 2063793]
# of segments with min 5 frames for single frame analysis  (13807, 2)


smoothing vecs: 100%|██████████| 9669/9669 [00:34<00:00, 283.35it/s]


vecs smooth:  (9669,)  e.g.:  (9, 2, 2)


Getting angles: 100%|██████████| 9669/9669 [00:01<00:00, 8916.32it/s] 


angles:  (9669,)  eg.g.  [ 0.          0.09724808  0.01394613 -0.04392371  0.02518353 -0.00601063
 -0.04032807  0.00200094  0.05324679]
...smoothing angles...


...

angles:  (9669,)  eg.g.  [ 0.         0.        39.9527359  0.         0.         0.
  0.         0.         0.       ]


Getting velocity and acceleration: 100%|██████████| 9669/9669 [00:00<00:00, 80546.75it/s]


DISCRETIZING 


100%|██████████| 9669/9669 [00:00<00:00, 111601.72it/s]


... DONE...


In [140]:
###########################################
###########################################
###########################################



In [47]:
####################################################
########### VISUALIZE DISCRETE BEHAVIORS ###########
####################################################

fname = '/media/cat/1TB/dan/cohort1/slp_files/2020_3_16_12_57_12_418305_compressed_animalID_3_single_frame_allData.npz'

visualize_discrete_behaviors(fname)

(78,) (78,) (78,)
(8,) (7,) (6,)
accel [0] [203.36943   88.59603   73.121414  75.251236   8.038322  97.590485]
(6, 2) [[0. 1.]
 [1. 1.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 1.]]


In [12]:
##################################
######### GET PCA DATA ###########
##################################

# 
(all_continuous,
all_discretized,
all_hist,
all_unique,
angles_thresh,
angles,
angles_discretized,
ang_unique,
ang_hist,
acc_thresh,
acc,
acc_hist,
acc_discretized,
acc_unique) =  compute_discretized_and_histograms(animal_id, 
                                  min_duration)

# 

agnels thresholds:  [[-100000000.0, -45], [-45, 45], [45, 100000000.0]]
accelaration thresholds:  [[0, 40], [40, 100000000.0]]
 FRAME IDS MISSING , UPGRADE DATAFILES 
ANIMAL Specific data:  (2069710, 6, 2)
# of headnose locations:  (521682,)
# of durations of at least  25 (229931, 2)


Getting egocentric vectors: 100%|██████████| 192964/192964 [00:13<00:00, 14623.29it/s]


vecs_ego:  (192964, 25, 2, 2)


Getting angles: 100%|██████████| 192964/192964 [00:17<00:00, 10868.13it/s]


...smoothing angles...


...

COMPUTING Klaus normalized histograms
BINS angles:  [-0.5  0.5  1.5  2.5]


100%|██████████| 192964/192964 [00:04<00:00, 40762.93it/s]


BINS accel:  [-0.5  0.5  1.5]


100%|██████████| 192964/192964 [00:04<00:00, 42213.15it/s]


Angles histograms:  (192964, 3)
Accel histograms:  (192964, 2)
all histograms:  (192964, 5)

Unique angles:  (292, 3)
Unique accel:  (24, 2)
# possible combinations:  7008
# of actual combinations:  (3527, 5)
ang unique:  [[0.   0.16 0.84]
 [0.   0.2  0.8 ]]
acc unique:  [[0.    1.   ]
 [0.043 0.957]]
... done


In [13]:
#print (angles[:100])

print (ang_hist)
plt.plot(np.hstack(angles[:1000]))

#plt.ylim(bottom=0)
plt.ylabel("Rotation (degrees/sec)")
plt.show()

[[0.2  0.8  0.  ]
 [0.2  0.76 0.04]
 [0.24 0.76 0.  ]
 ...
 [0.2  0.72 0.08]
 [0.16 0.68 0.16]
 [0.28 0.68 0.04]]


In [14]:
animal_ids = [0]
#min_duration = 50
plot_angle_acceleration_distributions(animal_ids, min_duration)

angles:  (192964, 25)



In [8]:
fps = 25
for k in range(angles.shape[0]):
    idx = np.where(np.abs(angles[k])>45)[0]
    if idx.shape[0]>0:
        print (k, np.round(angles[k],1))


0 [  0.    0.    0.  -48.6 -34.5 -51.7 -34.5   0.  -33.2   0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0. ]
1 [  0.    0.  -34.5 -34.5 -51.7 -34.5   0.  -33.2   0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   22.
   0. ]
2 [-51.7 -34.5 -34.5 -51.7 -34.5   0.  -33.2   0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0. ]
29 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.  57.1  0. ]
30 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0. -69. -64.]
31 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.  62.5 67.2]
33 [  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  -75.3

KeyboardInterrupt: 

In [15]:
import umap

#data = np.load('/home/cat/S_uniq_sym.npy')
#X_subsampled = data

X_subsampled = all_hist.copy()

print ("fitting umap on" ,X_subsampled.shape)
if False:
    noise = np.random.rand(X_subsampled.shape[0], X_subsampled.shape[1])/10.
    print ("noise mean: ", noise.mean())

    #for k in range(6):
    #    plt.subplot(2,3,k+1)
    #    
    #    
    #    
    print ("noise: ", noise.shape)
    X_subsampled += noise

umap = umap.UMAP(n_components=2,
                 n_neighbors=50,
                 min_dist = 0.1,
                 init='random',
                 random_state=0)

# 
X_fit = umap.fit_transform(X_subsampled)

# 
print (X_fit.shape)
plt.scatter(X_fit[:,0], X_fit[:,1],
            s=10,
            alpha=.5)
plt.show()


fitting umap on (192964, 5)
(192964, 2)


In [12]:
np.save('/home/cat/X_fit_all_histograms_updated5.npy', X_fit)

print (X_fit.shape)
plt.scatter(X_fit[:,0], X_fit[:,1],
            s=1,
            alpha=.1)
plt.show()


(264580, 2)


In [15]:
animal_ids = [0]
min_duration = 8
plot_angle_acceleration_distributions(animal_ids, min_duration)


angles:  (264580, 8)



In [7]:
###################################
######## GENERATE S MATRIX ########
###################################
#
print (ang_unique.shape)
print (acc_unique.shape)
print (all_unique.shape)      


(emd_angle_unique_norm, 
   emd_acc_unique_norm) = compute_emd_normalized_unique_angle_acc(ang_unique, acc_unique)


#     
if True:
    n_split = 200
    add_noise = True
    indexes_split = np.array_split(np.arange(all_unique.shape[0]),n_split)#[:32]
    parmap.map(compute_S_matrix_parallel, indexes_split,
               add_noise,
               all_unique, 
               ang_unique, acc_unique, n_split,
               min_duration, 
               emd_angle_unique_norm, 
               emd_acc_unique_norm,
               pm_processes= 8,
               pm_pbar = True)
      
else:
    res = []
    for k in range(all_unique.shape[0]):
        res.append(compute_S_matrix_parallel([k], all_unique, ang_unique, acc_unique,
                                   min_duration))


(35, 3)
(28, 3)
(518, 6)


203it [00:00, 290.53it/s]              


In [24]:
###########################################
###########################################
###########################################
# 
min_duration = 8
print (all_unique.shape)

S_uniq_sym = reconstruct_S_unique_from_files(all_unique, min_duration)
    
D_em = np.sqrt(-S_uniq_sym)

fig = plt.figure()
ax=plt.subplot(121)
plt.imshow(S_uniq_sym, aspect='auto')

ax=plt.subplot(122)
plt.imshow(D_em, aspect='auto')
plt.show()

(518, 6)


100%|██████████| 200/200 [00:00<00:00, 1747.80it/s]


In [28]:
import umap

#data = np.load('/home/cat/S_uniq_sym.npy')
#X_subsampled = data

X_subsampled = D_em.copy()

print ("fitting umap on" ,X_subsampled.shape)
if True:
    noise = np.random.rand(X_subsampled.shape[0], X_subsampled.shape[1])/10.
    print ("noise mean: ", noise.mean())

    #for k in range(6):
    #    plt.subplot(2,3,k+1)
    #    
    #    
    #    
    print ("noise: ", noise.shape)
    X_subsampled += noise

umap = umap.UMAP(n_components=2,
                 n_neighbors=20,
                 min_dist = 0.1,
                 init='random',
                 random_state=0)

# 
X_fit = umap.fit_transform(X_subsampled)

# 
print (X_fit.shape)
plt.scatter(X_fit[:,0], X_fit[:,1],
            s=10,
            alpha=.5)
plt.show()

fitting umap on (518, 518)
noise mean:  0.04994701993154144
noise:  (518, 518)
(518, 2)


In [38]:
############################################
############################################
############################################
x_flat = D_em.flatten()

print (D_em)
fig = plt.figure()
#y = np.histogram(x_flat, bins = np.arange(-.05,0,0.001))
y = np.histogram(x_flat, bins = np.arange(0, 0.5, 0.01))
plt.plot(y[1][1:],y[0])
plt.semilogy()
plt.show()

[[-0.          0.05558333  0.08333334 ...  0.05558333  0.11108334
   0.0555    ]
 [ 0.05558333 -0.          0.02775    ...  0.          0.0555
   0.02775   ]
 [ 0.08333334  0.02775    -0.         ...  0.          0.
   0.02783333]
 ...
 [ 0.05558333  0.          0.         ... -0.          0.0555
   0.02775   ]
 [ 0.11108334  0.0555      0.         ...  0.0555     -0.
   0.05558333]
 [ 0.0555      0.02775     0.02783333 ...  0.02775     0.05558333
  -0.        ]]


In [107]:
# from sklearn.cluster import AffinityPropagation
# import numpy as np

# X = S_uniq_sym

# clustering = AffinityPropagation(random_state=5).fit(X)

# print (clustering.labels_)



[201 200 199 ... 201 201 201]


In [9]:
# Compute and plot dendrogram.
import scipy.cluster.hierarchy as sch
fig = plt.figure()
axdendro = fig.add_axes([0.09,0.1,0.2,0.8])

# 
D = S_uniq_sym

# 
print ("Computing Y: ")
Y = sch.linkage(D, method='centroid')

print ("Computing Z: ")
Z = sch.dendrogram(Y, orientation='right')
axdendro.set_xticks([])
axdendro.set_yticks([])

# Plot distance matrix.
axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])
index = Z['leaves']
D = D[index,:]
D = D[:,index]
im = axmatrix.matshow(D, aspect='auto', origin='lower')
axmatrix.set_xticks([])
axmatrix.set_yticks([])

# Plot colorbar.
axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
plt.colorbar(im, cax=axcolor)

# Display and save figure.
fig.show()
fig.savefig('dendrogram.png')


Computing Y: 
Computing Z: 


In [114]:
print (Y.shape)

(5088, 4)


In [117]:
d = np.load('/media/cat/1TB/dan/cohort1/slp/2020_3_16_07_55_55_775234_compressed.npy')
print (d.shape)

(89985, 4, 14, 2)


In [98]:
#########################################
######## SUBSAMPLE DATA ###########
#########################################
# # 

# def compute_S_matrix2(all_histograms, subsample=10):
    
#     all_hist_sub = all_histograms[::subsample]
    
#     print ("all hist sub: ", all_hist_sub.shape)
    
#     s_matrix = np.zeros((all_hist_sub.shape[0], all_hist_sub.shape[0]), dtype=np.float32)
#     for i in trange(all_hist_sub.shape[0]):
#             for p in range(i+1, all_hist_sub.shape[0],1):
#                 s_matrix[i,p] = scipy.stats.wasserstein_distance(all_hist_sub[i], 
#                                                                  all_hist_sub[p])

#     ## normalize by n-1
#     #emd_angle = emd_angle / ang_unique.shape[1]

#     return s_matrix
    
# print (all_histograms.shape)

# s_matrix = compute_S_matrix2(all_histograms, subsample = 50)

# print ("S_matrix: ", s_matrix.shape)
# # (emd_angle_unique_norm, 
# #  emd_acc_unique_norm) = compute_emd_normalized_unique_angle_acc(ang_unique, 
# #                                                                 acc_unique)



In [11]:
import umap

#data = np.load('/home/cat/S_uniq_sym.npy')
#X_subsampled = data

X_subsampled = S_uniq_sym.copy()

print ("fitting umap on" ,X_subsampled.shape)
if False:
    noise = np.random.rand(X_subsampled.shape[0], X_subsampled.shape[1])/50000.
    print ("noise mean: ", noise.mean())

    #for k in range(6):
    #    plt.subplot(2,3,k+1)
    #    
    #    
    #    
    print ("noise: ", noise.shape)
    X_subsampled += noise

umap = umap.UMAP(n_components=2,
                 n_neighbors=20,
                 min_dist = 0.1,
                 init='random',
                 random_state=0)

# 
X_fit = umap.fit_transform(X_subsampled)

# 
print (X_fit.shape)
plt.scatter(X_fit[:,0], X_fit[:,1],
            s=10,
            alpha=.5)
plt.show()

fitting umap on (518, 518)
(518, 2)


In [31]:
X_fit = np.load('/home/cat/S_uniqu_sym_X_fit_0.1_200_original_17355Frames.npy')
print (X_fit.shape)

plt.scatter(X_fit[:,0], X_fit[:,1],
            s=10,
            alpha=1)
plt.show()

(17355, 2)


In [17]:
np.save('/home/cat/S_uniqu_sym_X_fit_0.1_200.npy', X_fit)

In [95]:
data = np.load('/home/cat/cluster.npz', allow_pickle=True)
locs = data['cluster_locs']
print ("len ", len(locs))

fig = plt.figure()
ax=plt.subplot(121)
idx = np.random.choice(np.arange(len(locs)), len(locs), replace=False)
for ctr,k in enumerate(idx):
    plt.scatter(locs[k][:,0],
                locs[k][:,1],
                color = 'blue')

ax=plt.subplot(122)

colors = plt.cm.gist_ncar(np.linspace(0,1,len(locs)))
idx = np.random.choice(np.arange(len(locs)), len(locs), replace=False)
for ctr,k in enumerate(idx):
    plt.scatter(locs[k][:,0],
                locs[k][:,1],
                color = colors[ctr])
plt.show()

len  39


In [36]:
from sklearn.cluster import AffinityPropagation
import numpy as np

# 
X1 = X_fit[::5] #[:30*18]
print ("X1: ", X1.shape)

X2 = scipy.spatial.distance.cdist(X1, X1, metric='cosine')
print ("X2: ", X2.shape)

clustering = AffinityPropagation(affinity='precomputed', 
                                 preference=-1.1,
                                 random_state=5, 
                                 verbose=True).fit(X2)

#
print ("FINISHED: ", np.unique(clustering.labels_))

# 
fig = plt.figure()
plt.scatter(X_fit[:,0], X_fit[:,1],
            c='red',
            edgecolor='black',
            alpha=.1)

labels = clustering.labels_
plt.scatter(X1[:,0], X1[:,1],
            c=labels,
            edgecolor='black',
            alpha=1)

plt.show()


X1:  (5449, 2)
X2:  (5449, 5449)
Converged after 96 iterations.
FINISHED:  [0 1 2 3]


In [6]:
#####################################
# print ("all data: ", all_unique.shape)

# # 
# def cleanup_data(all_unique, min_occurance = 10, max_occurance = 1E4):

#     all_clean = all_unique.copy()
    
#     frames_unique, indexes, counts_all  = np.unique(all_unique, axis=0, 
#                                                 return_counts=True, 
#                                                 return_index=True)
    
#     # 
#     print ("uinque feature vectors: ", frames_unique.shape, " Counts: ", counts_all.shape, "  all indexes: ", indexes.shape)
#     print ("unique features: ", frames_unique)
#     print ("counts_all:", counts_all)
#     print ("indexes: ", indexes)
    
#     # 
#     cts_unique = np.unique(counts_all)#[::-1]
#     print (" unique counts: ", cts_unique)

#     ##################################
#     ##################################
#     ##################################
#     for c in cts_unique:
#         idx3 = np.where(counts==c)[0]
#         n_counts.append(idx3.shape[0])

#         if c<min_occurance or c>max_occurance:
#             id4 = np.where(counts_all==c)[0]
#             print (" # of unique frames that occur ", c, " times is: ", idx3.shape)
#             #print ("ID4: ", id4)
            
#             for id_ in id4:
#                 frame = frames_unique[id_]

#                 id4 = np.sum(np.equal(all_clean, frame),axis=1)
#                 id4 = np.where(id4==len(frame))[0]

#                 all_clean = np.delete(all_clean,id4,axis=0)


#     ##################################
#     ##################################
#     ################################## 
#     frames_unique, indexes, counts_all  = np.unique(all_clean, axis=0, 
#                                                     return_counts=True, 
#                                                     return_index=True)
    
#     # 
#     print ("uinque feature vectors: ", frames_unique.shape, " Counts: ", counts_all.shape, "  all indexes: ", indexes.shape)
#     print ("unique features: ", frames_unique)
#     print ("counts_all:", counts_all)
#     print ("indexes: ", indexes)
    
#     # 
#     cts_unique = np.unique(counts_all)#[::-1]
    
    
    
#     # 
#     return all_clean, cts_unique

# ##############################################
# ##############################################
# ##############################################
# min_occurance = 1
# max_occurance = 100
# all_clean, cts_unique = cleanup_data(all_unique, min_occurance, max_occurance)

# print("ALL CLENA: ", all_clean.shape)

# plt.plot(cts_unique)
# plt.xlabel("Number of frames")
# plt.ylabel("Number of occurances")
# #plt.xlim(left=-100)
# plt.ylim(bottom=1)
# plt.semilogy()
# plt.semilogx()
# plt.show()

In [ ]:
# EDM CALCULATIONS BELOW

In [7]:
##########################################################
######## COMPUTE EMD ON UNIQUE FEATURE WISE PAIRS ########
########################################################## 
emd_angle_unique_norm, emd_acc_unique_norm = compute_emd_normalized_unique_angle_acc(ang_unique, acc_unique)


NameError: name 'ang_unique' is not defined

(325, 3)
(706, 4)
(17355, 7)


203it [23:53,  7.06s/it]                          


(17355, 17355)


/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:446: UserWarning: Warning: converting a masked element to nan.
  dv = np.float64(self.norm.vmax) - np.float64(self.norm.vmin)
/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:453: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/home/cat/anaconda3/envs/gerbil/lib/python3.6/site-packages/matplotlib/image.py:458: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)


In [27]:
####################################################
######## RECONSTRUCT AND SYMMETRIZE S_UNIQUE #######
####################################################

def reconstruct_S_unique(res, all_unique, indexes_split):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    fname_out = os.path.join(root_dir, "S_unique_symmetric.npy")
    
    S_unique = np.zeros((all_unique.shape[0],all_unique.shape[0]),'float32') #+np.nan

    for k in range(len(indexes_split)):
        S_unique[indexes_split[k]] = res[k]

    print (S_unique.shape)

    S_unique_symmetric = S_unique + S_unique.T - np.diag(np.diag(S_unique))

    np.save(fname_out, S_unique_symmetric)
    
    return S_unique_symmetric


def reconstruct_S_unique_from_files(all_unique):

    root_dir = '/media/cat/1TB/dan/cohort1/slp/'
    fnames = glob.glob(os.path.join(root_dir, "S_unique")+"/*.npz")
    
   
    #S_unique = np.zeros((all_unique.shape[0],all_unique.shape[0]),'float32') #+np.nan
    S_unique = np.zeros((17355,
                         17355),'float32') #+np.nan

    for fname in tqdm(fnames):
        # 
        data = np.load(fname, allow_pickle=True)
        
        # 
        S = data['S']
        indexes = data['indexes']
        
        #
        S_unique[indexes] = S

    #     
    S_unique_symmetric = S_unique + S_unique.T - np.diag(np.diag(S_unique))
     
    return S_unique_symmetric


# 
all_unique = None
S_uniq_sym = reconstruct_S_unique_from_files(all_unique)

    
plt.imshow(S_uniq_sym, aspect='auto')
plt.show()

100%|██████████| 200/200 [00:03<00:00, 60.03it/s]


In [25]:
#################################
######## CLUSTER S_UNIQUE #######
#################################

from sklearn.cluster import AffinityPropagation
import numpy as np
# X = np.array([[1, 2], [1, 4], [1, 0],
#               [4, 2], [4, 4], [4, 0]])

X = S_uniq_sym #[:30*18]
# print ("X: ", X.shape)
# clustering = AffinityPropagation(random_state=5).fit(X)

# #
#clustering

#
cluster_ids = np.unique(clustering.labels_)
print ("# clusters: ", np.unique(cluster_ids))

# 
S_unique_clustered = np.zeros(X.shape, 'float32')

ctr=0
for c in cluster_ids:
    idx = np.where(clustering.labels_==c)[0]
    print (c,"idx: ", idx.shape)
    S_unique_clustered[ctr:ctr+idx.shape[0]] = S_uniq_sym[idx]
    ctr+=idx.shape[0]

    
# ax2=plt.subplot(121)
# plt.imshow(X, aspect='auto')



# ax2=plt.subplot(122)
# plt.imshow(S_unique_clustered, aspect='auto')
# plt.show()

# 
# clustering.predict([[0, 0], [4, 4]])

# 
#print (clustering.cluster_centers_)


# clusters:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 2

In [40]:
# ###############################################
# ###############################################
###############################################
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_fit[:,0], X_fit[:,1], X_fit[:,2],
           c = clustering.labels_)
plt.show()


In [ ]:
################################
######## PREDICT S_ALL #########
################################





In [ ]:
################################################
######## VISUALIZE S_ALL + CLUSTER IDS #########
################################################





In [ ]:
##########################################
##########################################
##########################################

# #
# res = hist_rows
# for k in trange(hist_rows.shape[0]):
#     for p in range(k+1, hist_rows.shape[0],1):
#         temp1 = hist_rows[k]
#         temp2 = hist_rows[p]
#         #print ("temp1: ", temp1. shape, temp2.shape)
#         res = scipy.stats.wasserstein_distance(temp1, 
#                                                temp2)
        
        
def edm_parallel(idx, data_in):
    
    res = []
    for i in idx:
        for p in range(i+1, hist_rows.shape[0],1):
            res.append(scipy.stats.wasserstein_distance(hist_rows[i], 
                                                        hist_rows[k]))
    return res
                  
#                       
import parmap
                       
if True:
    idx_split = np.array_split(np.arange(hist_rows.shape[0]),100)
    res = parmap.map(edm_parallel, 
                     idx_split, 
                     hist_rows[:,:3],
                     pm_processes= 8,
                     pm_pbar=True)
    
    
                       

In [9]:

data = np.load('/media/cat/1TB/dan/cohort2/2020_07_21_11_26_07_855478_compressed.npy')
print (data.shape)

data_fixed = np.load('/media/cat/1TB/dan/cohort2/2020_07_21_11_26_07_855478_compressed_fixed.npy')
print (data_fixed.shape)

animal_id=0
feature = 0
plt.plot(data[:,animal_id,feature,0], c='blue')
plt.plot(data_fixed[:,animal_id,feature,0]+100, c='red')
    
plt.show()


(28802, 6, 6, 2)
(28802, 6, 6, 2)


In [280]:
# okr stimulus protocol

n_directions = 8
duration_on = 5
duration_off = 5
repeats = 10

total_time = (duration_on+duration_off)*repeats*n_directions

print ("total time: ", total_time)




total time:  800
